In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Mon Jul  7 14:00:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
# 📌 1. 기본 라이브러리
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, f1_score
from xgboost import XGBClassifier
from collections import Counter

In [ ]:
# 📌 2. 데이터 불러오기
train = pd.read_csv('C:/Users/PC/Desktop/부트캠프/workspace/FinalProject/주피터 파일/피처 선택_43개/피처 선택/43_Segment 추가.csv')

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/피처 선택/test_피처43.csv')

In [ ]:
# ▶ 1. 피처(X), 정답(y) 분리
X_vif = train.drop(columns=['Segment'])
y_vif = train['Segment']

# ▶ 2. 필요 시 레이블 인코딩 (CatBoost는 문자형도 처리 가능하지만 교차검증 편의 위해 사용)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_vif_encoded = le.fit_transform(y_vif)

In [ ]:
# 'ID' 컬럼 제거
X_vif = X_vif.drop(columns=['ID'], errors='ignore')

In [ ]:
X_vif.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 43 columns):
 #   Column              Dtype
---  ------              -----
 0   입회일자_신용             int64
 1   탈회횟수_발급1년이내         int64
 2   _1순위카드이용건수          int64
 3   _2순위카드이용금액          int64
 4   최종카드발급경과월           int64
 5   CA한도금액              int64
 6   월상환론한도금액            int64
 7   카드이용한도금액_B2M        int64
 8   이용금액_일시불_R12M       int64
 9   이용금액_할부_무이자_R12M    int64
 10  이용금액_체크_R12M        int64
 11  최대이용금액_일시불_R12M     int64
 12  최대이용금액_체크_R12M      int64
 13  이용개월수_할부_무이자_R3M    int64
 14  쇼핑_도소매_이용금액         int64
 15  _1순위업종_이용금액         int64
 16  _2순위업종_이용금액         int64
 17  _3순위업종_이용금액         int64
 18  _2순위쇼핑업종_이용금액       int64
 19  _3순위쇼핑업종_이용금액       int64
 20  _1순위교통업종_이용금액       int64
 21  할부금액_무이자_3M_R12M    int64
 22  이용금액_오프라인_R6M       int64
 23  이용건수_오프라인_R6M       int64
 24  이용금액_오프라인_R3M       int64
 25  이용금액_오프라인_B0M       int64
 26  이용금액_A페이_R3M  

In [ ]:
# ▶ CatBoost 설치
!pip install -q catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 24.6 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np
import warnings

warnings.filterwarnings('ignore')

# ▶ 교차검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
f1_scores = []

# ▶ CatBoost 파라미터
model_params = {
    "loss_function": "MultiClass",      # 다중 클래스용
    "eval_metric": "MultiClass",
    "task_type": "GPU",
    "learning_rate": 0.01,
    "iterations": 70000,
    "l2_leaf_reg": 50,
    "random_seed": 42,
    "od_type": "Iter",
    "depth": 5,
    "early_stopping_rounds": 15000,
    "border_count": 64,
    "verbose": 0
}

# ▶ 교차검증 반복
for fold, (train_idx, val_idx) in enumerate(cv.split(X_vif, y_vif_encoded)):
    print(f"▶ Fold {fold + 1}")

    X_train, X_val = X_vif.iloc[train_idx], X_vif.iloc[val_idx]
    y_train, y_val = y_vif_encoded[train_idx], y_vif_encoded[val_idx]

    train_pool = Pool(data=X_train, label=y_train)
    val_pool = Pool(data=X_val, label=y_val)

    model = CatBoostClassifier(**model_params)
    model.fit(train_pool, eval_set=val_pool, use_best_model=True)

    y_pred = model.predict(X_val)
    f1 = f1_score(y_val, y_pred, average='micro')
    f1_scores.append(f1)

    print(f"✅ F1_micro: {f1:.5f}")

# ▶ 평균 F1 출력
print(f"\n🎯 평균 F1_micro (5 folds): {np.mean(f1_scores):.5f}")

▶ Fold 1
✅ F1_micro: 0.91661
▶ Fold 2
✅ F1_micro: 0.91600
▶ Fold 3
✅ F1_micro: 0.91645
▶ Fold 4
✅ F1_micro: 0.91704
▶ Fold 5
✅ F1_micro: 0.91661

🎯 평균 F1_micro (5 folds): 0.91654


In [ ]:
# ▶ cat_features 정의 (범주형...없긴 함)
cat_features = X_vif.select_dtypes(include='object').columns.tolist()

In [ ]:
import pickle
import pandas as pd
import numpy as np

# ▶ 저장 경로 지정
base_path = '/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/catboost'

# ▶ 1. Fold별 f1_micro 점수 저장
with open(f'{base_path}/f1_scores.pkl', 'wb') as f:
    pickle.dump(f1_scores, f)

# ▶ 2. 평균 f1_micro 점수 저장
with open(f'{base_path}/f1_mean.txt', 'w') as f:
    f.write(str(np.mean(f1_scores)))

# ▶ 3. LabelEncoder 저장 (Segment 인코딩 복원용)
with open(f'{base_path}/label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)

# ▶ 4. 학습용 피처(X_vif)와 타깃(y_vif_encoded) 저장
X_vif.to_csv(f'{base_path}/X_vif.csv', index=False)
pd.Series(y_vif_encoded, name='Segment_encoded').to_csv(f'{base_path}/y_vif_encoded.csv', index=False)

# ▶ 5. 범주형 컬럼 리스트 저장
with open(f'{base_path}/cat_features.pkl', 'wb') as f:
    pickle.dump(cat_features, f)

# ▶ 6. 최종 모델 저장 (선택)
# final_model.save_model(f'{base_path}/final_catboost_model.cbm')


In [ ]:
# from xgboost import XGBClassifier
# from sklearn.model_selection import StratifiedKFold, cross_val_score
# from sklearn.metrics import make_scorer, f1_score

# # 클래스 비율 기반 sample_weight 계산
# # → 이미 y_encoded 가 숫자형이면 그대로 사용
# from collections import Counter

# class_counts = Counter(y_encoded)
# total_count = sum(class_counts.values())
# class_weights = {cls: total_count / count for cls, count in class_counts.items()}

# # 각 샘플에 대한 가중치 부여
# sample_weights = np.array([class_weights[cls] for cls in y_encoded])

# # XGBoost 모델 정의 (GPU 사용)
# model = XGBClassifier(
#     objective='multi:softprob',
#     num_class=5,  # 클래스 수 (A~E)
#     tree_method='gpu_hist',  # ✅ GPU 사용
#     eval_metric='mlogloss',
#     use_label_encoder=False,
#     n_jobs=-1,
#     random_state=42
# )

# # Stratified K-Fold 설정
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# # f1_micro scorer 정의
# f1_micro = make_scorer(f1_score, average='micro')

# # 교차검증 수행 (sample_weight 사용)
# scores = cross_val_score(
#     model,
#     X, y_encoded,
#     cv=cv,
#     scoring=f1_micro,
#     fit_params={'sample_weight': sample_weights}
# )

# print(f"XGBoost f1_micro 평균 점수: {scores.mean():.4f}")